In [ ]:
from functools import partial
from pathlib import Path

import geopandas
import numpy
import pandas
import rasterio
import orjson as json
from snail.intersection import get_cell_indices
from tqdm.auto import tqdm

tqdm.pandas()

In [ ]:
base_dir = Path("../results")

In [ ]:
damage_cost = geopandas.read_parquet(base_dir/"africa-latest_filter-road-tertiary/hazard-landslide-arup/damage_EAD_and_cost_per_trigger.geoparquet")

In [ ]:
damage_cost.to_file("africa-latest_damage_EAD_and_cost_per_trigger.gpkg")

In [ ]:
splits_dir = base_dir / 'direct_damages/africa-latest_filter-road-tertiary/hazard-landslide-arup/split_EAD_and_cost_per_trigger'


In [ ]:
dfs = [geopandas.read_parquet(fname) for fname in splits_dir.glob("*.geoparquet")]

splits = pandas.concat(dfs)

In [ ]:
splits.to_file("split_EAD_and_cost_per_trigger.gpkg", driver="GPKG", engine="pyogrio")

In [ ]:
splits.columns

In [ ]:
with rasterio.open(base_dir/"input/hazard-landslide-arup/africa-latest/ls_eq_tiled.tif") as src:
    pass

src.width, src.height, src.transform

In [ ]:
raster_height = src.height
raster_width = src.width
raster_transform = src.transform
data_col = 'hazard-_landslide_sum__road_damage_fraction_EAD'

data = numpy.zeros((raster_height, raster_width))

splits_df = splits

def get_cell_indices_kw(geom, raster_height=0, raster_width=0, raster_transform=None):
    return get_cell_indices(geom, raster_height, raster_width, raster_transform)

cell_indices_of_split_geometry = partial(get_cell_indices_kw, raster_height=src.height, raster_width=src.width, raster_transform=list(src.transform))

splits_df = pandas.concat(
    [
        splits_df[[data_col]],
        splits_df.geometry.progress_apply(cell_indices_of_split_geometry)
    ],
    axis="columns"
).rename(columns={"geometry":"cell_index"})

In [ ]:
value_per_cell = splits_df[['cell_index', data_col]].groupby('cell_index').sum()

In [ ]:
for _, item in value_per_cell.reset_index().iterrows():
    col, row = item.cell_index
    data[int(row), int(col)] = item[data_col]


In [ ]:
with rasterio.open(
        'africa-latest_damage_EAD.tif',
        'w',
        driver='GTiff',
        height=data.shape[0],
        width=data.shape[1],
        count=1,
        dtype=data.dtype,
        crs='+proj=latlong',
        transform=raster_transform,
        compress='lzw'
    ) as dataset:
    dataset.write(data, 1)

# Demo options

In [ ]:
list(Path(".").glob("*.gpkg"))

In [ ]:
basins = geopandas.read_file('selection__hybas_lev12_v1c.gpkg')
minx, miny, maxx, maxy = basins.centroid.total_bounds

In [ ]:
all_basins = geopandas.read_parquet("../results/input/hydrobasins/hybas_lev12_v1c.geoparquet")

In [ ]:
basins = all_basins.cx[minx:maxx, miny:maxy]

In [ ]:
splits = geopandas.read_file('intersection__ls_nbs_current__split_ead.gpkg').drop(columns="DN").drop_duplicates()
splits.shape

In [ ]:
hazard_cols = [c for c in splits.columns if "EAD" in c]
hazard_cols

In [ ]:
options = geopandas.read_file('extract__ls_nbs_current.gpkg').query("DN != 0").reset_index(drop=True)  # in case we polygonised some holes, drop DN==0
print(options.shape)
options["feature_id"] = range(len(options))
options["option_landuse"] = options["DN"].map({1: "crops", 2: "other"})
options = (
    options
    .sjoin(basins[["HYBAS_ID", "geometry"]], predicate="intersects")
    .drop(columns=["index_right", "DN"])
    .drop_duplicates(subset="feature_id")
    .sort_values(by="feature_id")
)
buf_geom = options.geometry.to_crs("ESRI:54009").buffer(500).to_crs("EPSG:4326")
buf = options.copy()
buf.geometry = buf_geom

In [ ]:
options

In [ ]:
options.iloc[0]

In [ ]:
joined = buf.sjoin(splits, how="left")
options_damages = joined[["feature_id"] + hazard_cols].groupby("feature_id").sum()

def json_nanlist(series):
    """Aggregation function, returns a JSON-encoded list of unique values
    from the series (excluding NaNs) decoded to str (assuming json.dumps
    is returning bytes, as in orjson)
    """
    return json.dumps(list(series.dropna().unique())).decode()

options_ids = joined[["feature_id", "id"]].groupby("feature_id").agg({"id": json_nanlist}).rename(columns={"id": "feature_ids"})

options_ead = (
    options
    .set_index("feature_id")
    [["option_landuse", "HYBAS_ID", "geometry"]]
    .join(options_ids)
    .join(options_damages)
).reset_index()

options_ead["area_m2"] = options_ead.geometry.to_crs("ESRI:54009").area
options_ead["area_ha"] = options_ead["area_m2"] * 1e-4
# On average, self-planted projects cost the landowner 40-80p per tree plus VAT,
# or £1-£1.80 per tree plus VAT where planted by a Trust-arranged contractor. A
# 1ha site at the recommended 1,000-1,600 trees per hectare would therefore cost
# the landowner around £900-2,000.
# https://www.woodlandtrust.org.uk/plant-trees/trees-for-landowners-and-farmers/morewoods/#:~:text=On%20average%2C%20self%2Dplanted%20projects,landowner%20around%20%C2%A3900%2D2%2C000.
# As of 2024-11: 900 GBP is ~1160 USD
#               2000         2580
options_ead["cost_usd_amin"] = options_ead["area_ha"] * 1160  # demo cost_per_ha
options_ead["cost_usd_mean"] = options_ead["area_ha"] * ((1160 + 2580) / 2)  # demo cost_per_ha
options_ead["cost_usd_amax"] = options_ead["area_ha"] * 2580  # demo cost_per_ha
# On average, one hectare of native broadleaf woodland will store 300 - 350
# tonnes of carbon over a 100-year period.
# https://www.woodlandtrust.org.uk/plant-trees/woodland-carbon-farmers-and-landowners/#:~:text=How%20much%20carbon%20do%20trees,over%20a%20100%2Dyear%20period.
options_ead["carbon_capture_t_amin"] = options_ead["area_ha"] * 300  # demo carbon benefit
options_ead["carbon_capture_t_mean"] = options_ead["area_ha"] * 325  # demo carbon benefit
options_ead["carbon_capture_t_amax"] = options_ead["area_ha"] * 350  # demo carbon benefit


options_ead.rename(columns={
    "hazard-ls_eq_tiled__road_damage_fraction_EAD": "hazard-ls_eq__avoided_ead_amin",
    "hazard-ls_eq_tiled__road_lower_EAD": "hazard-ls_eq__avoided_ead_mean",
    "hazard-ls_eq_tiled__road_upper_EAD": "hazard-ls_eq__avoided_ead_amax",
    "hazard-LS_RF_Median_1980-2018__road_damage_fraction_EAD": "hazard-ls_rf__avoided_ead_amin",
    "hazard-LS_RF_Median_1980-2018__road_lower_EAD": "hazard-ls_rf__avoided_ead_mean",
    "hazard-LS_RF_Median_1980-2018__road_upper_EAD": "hazard-ls_rf__avoided_ead_amax",
    "hazard-_landslide_sum__road_damage_fraction_EAD": "hazard-ls_sum__avoided_ead_amin",
    "hazard-_landslide_sum__road_lower_EAD": "hazard-ls_sum__avoided_ead_mean",
    "hazard-_landslide_sum__road_upper_EAD": "hazard-ls_sum__avoided_ead_amax",
}, inplace=True)
options_ead.iloc[11]

In [ ]:
options_ead.to_file("joined__ls_nbs_current__split_ead.geojson")
options_ead.to_file("joined__ls_nbs_current__split_ead.gpkg")